In [65]:
import onnxruntime
from scipy.special import softmax
from PIL import Image
from torchvision.transforms import ToTensor
import os
import numpy as np
import tensorflow as tf

In [ ]:
!pip install onnxruntime
!pip install git+https://github.com/onnx/tensorflow-onnx

In [ ]:
!pip install -U tf2onnx

In [4]:
!python -m tf2onnx.convert --saved-model /content/drive/MyDrive/CV_project/안구질환_모델/Resnet_squared_all_정상_아픔.tf --output model.onnx --opset 12

2024-06-03 05:40:56.233548: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-03 05:40:56.233654: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-03 05:40:56.237593: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-03 05:40:56.255222: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-03 05:40:57.719367: W tensorflow/compiler/tf2

In [7]:
!cp /content/model.onnx /content/drive/MyDrive/CV_project/안구질환_모델/

## onnx 추론


In [74]:
onnx_path = '/content/drive/MyDrive/CV_project/안구질환_모델/Resnet_squared_all_정상_아픔.tf/ResNet50v2_binary.onnx'

In [75]:
ort_session = onnxruntime.InferenceSession(onnx_path)
input_name = ort_session.get_inputs()[0].name

In [76]:
tf_path = '/content/drive/MyDrive/CV_project/안구질환_모델/Resnet_squared_all_정상_아픔.tf'
tf_model = tf.keras.models.load_model(tf_path)

In [82]:
sick_img_path = "/content/drive/MyDrive/CV_project/data/안구질환/개_train_test_val_squared/test/궤양성각막질환/crop_D0_125a8627-60a5-11ec-8402-0a7404972c70.jpg"

normal_img_path = "/content/drive/MyDrive/CV_project/data/안구질환/개_train_test_val_squared/test/정상/crop_D0_03b2ab5b-60a5-11ec-8402-0a7404972c70.jpg"

img_path = sick_img_path


In [83]:
img = Image.open(img_path)
img = img.resize((224, 224))
img = np.array(img, dtype=np.float32)

img = img / 255.
img = img.reshape((1, 224, 224, 3))
ort_inputs = {input_name: img}
ort_session.run(None, ort_inputs)

[array([[2.4139881e-05]], dtype=float32)]

In [84]:
img = Image.open(img_path)
img = img.resize((224, 224))
img = np.array(img)

img = img / 255.
img = tf.expand_dims(img, axis=0)
tf_model.predict(img, verbose=0)

array([[2.410668e-05]], dtype=float32)